## API lab

### Introduction:

While many data sets are provided though downloadable files, some are provided through application programming interface (API) which allows the code to access some data without downloading the entire dataset.

APIs provide sets of functions and procedures that allow accessing the features or data from an application or other service.


#### REST API
REST is essentially a set of useful conventions for structuring a web API. 'Web API' means an API that you interact with over HTTP, making requests to specific URLs, and often getting relevant data back in the response.

### Uses
- Scraping real time information - useful for example for accessing travel route info from mapping applications, social media feed. And any other data that being updated in real time or is changing quickly. Another example of this is stock price data. It doesn’t really make sense to regenerate a data set and download it every minute 
- Getting only the desired tables through filtering instead of downloading the entire data sets. Reddit comments are one example. It doesn’t make much sense (plus won't be permitted) to download the entire Reddit database if we want only some information

### General Terminology

#### Types of requests
characterize what action we are going to take by referring to the API

- GET: retrieve information (like search results). This is the most common type of request. Using it, we can get the data we are interested in from those that the API is ready to share.
- POST: adds new data to the server. Using this type of request, you can, for example, add a new item to your inventory.
- PUT: changes existing information. For example, using this type of request, it would be possible to change the color or value of an existing product.
- DELETE: deletes existing information

#### Status codes
briefly describe the result of the call.

- 200 – OK. The request was successful. The answer itself depends on the method used (GET, POST, etc.) and the API specification.
- 204 – No Content. The server successfully processed the request and did not return any content.
- 301 – Moved Permanently. The server responds that the requested page (endpoint) has been moved to another address and redirects to this address.
- 400 – Bad Request. The server cannot process the request because the client-side errors (incorrect request format).
- 401 – Unauthorized. Occurs when authentication was failed, due to incorrect credentials or even their absence.
- 403 – Forbidden. Access to the specified resource is denied.
- 404 – Not Found. The requested resource was not found on the server.
- 500 – Internal Server Error. Occurs when an unknown error has occurred on the server.

#### Endpoints
Usually, an Endpoint is a specific address (for example, https://weather-in-london.com/forecast), by referring to which you get access to certain features/data (in this example above – the weather forecast for London). Commonly, the name (address) of the endpoint corresponds to the functionality it provides.

### HereMap API
Provides mapping, location data and routing information services worldwide. Routing info is available for transit, walking, driving, bike modes.


In [5]:
import pandas as pd
import glob #Unix style pathname pattern expansion
import numpy as np
from dateutil import parser #powerful extensions to datetime; offers a generic date/time string parser
import time
from tqdm import tqdm #Customisable progressbar for iterators
try: #spelling depends on enviroment version 
    import urllib2 as urllib #URL handling module
except ImportError:
    import urllib.request as urllib
import json
import geopandas as gpd
import sys
from IPython.display import clear_output #Clears the output of the current cell receiving output
import requests #the module for making HTTP requests in Python; provides GET funcionality

#### Request API key by creating free account 

https://developer.here.com/sign-up?create=Freemium-Basic&keepState=true&step=account

(We recommend not to put your credit card info, all the API services will work for free without having to put your card)

1. After signing in, click on 'Start with REST APIs' part. Follow the instructions in the 'Access Manager' to create the app. 
2. After setting up the app, create the API key in the 'Credentials' section.

Note: For some API services, you may need an additional step to create a 'Project'. Please follow the below instructions to create a project and link your app to it. (not needed for this notebook/assignment)

1. Open the Projects Manager from the launcher.
2. Click Create new project and name the project.
3. In the 'Services' section, search for the services you'd like to access with the API. (eg. Maps services)
4. Link your app from the "Access Manager" tab to the above project and start using the API services.


#### Example 1: getting driving time

#### API endpoint
https://developer.here.com/documentation/routing-api/8.3.1/dev_guide/topics/send-request.html

#### Sample request

An API request needs an endpoint link with API key to fetch data. In some cases, you may also need appID and appCode to make a request. When you make a account in a particular API service, you will be asked to create and name your app. When you finish this step, you will be provided with a unique API key together with appID and appCode as a unique identifier for your app.

Please not that various APIs have certain restrictions on their usage. Please make sure to read the the usage terms before making acccount (especially if you are asked to enter your payment info beforehand!)

HereMap API has 250k free requests per month. (And it does not ask for credit card info when you create an account)

Also, please note the below example uses an API key for this lab's demonstration purposes only. Please create your HereMap account and get your unique API key when attempting the lab or homeworks yourself.

retreive a driving route between the two given points

In [6]:
# enter your api key from HereMap
apiKey = ''

lon_pickup, lat_pickup = -73.972136, 40.744301 ##origin: A point in Manhattan (around E 35th street)
lon_dropoff, lat_dropoff = -74.172945, 40.680108 # destination: Newark airport

#### Routing API info and documentation
https://developer.here.com/documentation/routing-api/dev_guide/topics/use-cases/calculate-route.html

Note the driving API endpoint is 'https://router.hereapi.com/v8/routes?transportMode=car&origin={},{}&destination={},{}&return=summary&apikey={}', which essentially gives the driving directions from origin to destination together with a summary of the whole trip.

For the above origin-destination lat-longs, the endpoint will look like

'https://router.hereapi.com/v8/routes?transportMode=car&origin=40.744301,-73.972136&destination=40.680108,-74.172945&return=summary&apikey={}'

Basically, the origin lat-long go in after the 'waypoint0=geo!' field and destination lat-long go in after 'waypoint1=geo!' field. Instead of manually putting these lat-long in the code, the pythonic way to do this is by the 'format' function. Wherever we need to insert variables, we put '{}' in the url and then specify the variables in their respective order of appearing in the url in the 'format' command.



In [7]:
url = str('https://router.hereapi.com/v8/routes?transportMode=car&origin={},{}&destination={},{}&return=summary&apikey={}'.format(
    lat_pickup, lon_pickup, lat_dropoff, lon_dropoff, apiKey))
print(url)

https://router.hereapi.com/v8/routes?transportMode=car&origin=40.744301,-73.972136&destination=40.680108,-74.172945&return=summary&apikey=TJqaLdkovbaRdGPPCOmO8egBGJALf56akxUCHJzJmM4


Now we'll make a request with the below command

In [8]:
data = urllib.urlopen(url).read().decode('utf-8')
data = json.loads(data)

The fetched data gives driving directions together with a summary of the whole trip

In [9]:
data

{'routes': [{'id': '7d147faa-7c2b-4273-94c1-a37d4b6ebc7b',
   'sections': [{'id': '52eb2e8a-be7e-4c16-8abf-311004ecfb06',
     'type': 'vehicle',
     'departure': {'time': '2023-09-21T01:40:34-04:00',
      'place': {'type': 'place',
       'location': {'lat': 40.7441211, 'lng': -73.9722708},
       'originalLocation': {'lat': 40.744301, 'lng': -73.9721361}}},
     'arrival': {'time': '2023-09-21T02:12:44-04:00',
      'place': {'type': 'place',
       'location': {'lat': 40.6790562, 'lng': -74.1703941},
       'originalLocation': {'lat': 40.6801079, 'lng': -74.172945}}},
     'summary': {'duration': 1930, 'length': 24876, 'baseDuration': 1707},
     'transport': {'mode': 'car'}}]}]}

The retrived output contains all information in the 'routes' key, including the travel time ('duration') and travel distance ('length')


Let's print the time and distance of the whole trip

In [10]:
# get travel time in seconds
data['routes'][0]['sections'][0]['summary']['duration']


1930

In [11]:
# travel distance in meters
data['routes'][0]['sections'][0]['summary']['length']

24876

#### Example 2: getting transit time

#### API info and documentation
https://developer.here.com/documentation/public-transit/dev_guide/quick-start.html


In [12]:
lon_pickup, lat_pickup = -73.972136, 40.744301
lon_dropoff, lat_dropoff = -74.008704, 40.708592

# API endpoint link
url = str('https://transit.router.hereapi.com/v8/routes?apiKey={}&origin={},{}&destination={},{}'.format(apiKey, lat_pickup, lon_pickup, 
                                                                                                        lat_dropoff, lon_dropoff))

data = urllib.urlopen(url).read().decode('utf-8')
data = json.loads(data)
data

{'routes': [{'id': 'R0',
   'sections': [{'id': 'R0-S0',
     'type': 'pedestrian',
     'departure': {'time': '2023-09-21T01:40:00-04:00',
      'place': {'type': 'place',
       'location': {'lat': 40.744301, 'lng': -73.972136}}},
     'arrival': {'time': '2023-09-21T01:57:00-04:00',
      'place': {'name': '33 St',
       'type': 'station',
       'location': {'lat': 40.746081, 'lng': -73.982076},
       'id': '717080436'}},
     'transport': {'mode': 'pedestrian'}},
    {'id': 'R0-S1',
     'type': 'transit',
     'departure': {'time': '2023-09-21T01:57:00-04:00',
      'place': {'name': '33 St',
       'type': 'station',
       'location': {'lat': 40.746081, 'lng': -73.982076},
       'id': '717080436'},
      'delay': 0},
     'arrival': {'time': '2023-09-21T02:10:00-04:00',
      'place': {'name': 'Fulton St',
       'type': 'station',
       'location': {'lat': 40.710368, 'lng': -74.009509},
       'id': '717080322'},
      'delay': 0},
     'transport': {'mode': 'subway',
    

Note that retrieved output keys are different from what we obtained in case of driving direction request. 

Here all data is present in the 'routes' key with information regarding intermediate stations. If we are interested in overall travel time, we'll need to access the first and last 'routes' and get the difference between the last arrival time and first departure time.

Important: You may get the retrieved repsonse other than above if any transit route is not available from specified origin to destination. 
Invalid responses are 'Timestamp out of validity period' and 'No result found'

In [13]:
# get travel time - subtract arrival time from last route and departure time from first route

(parser.isoparse(data['routes'][0]['sections'][-1]['arrival']['time']) 
    - parser.isoparse(data['routes'][0]['sections'][0]['departure']['time']))

datetime.timedelta(seconds=2040)

In [14]:
# convert to minutes

str(parser.isoparse(data['routes'][0]['sections'][-1]['arrival']['time']) 
    - parser.isoparse(data['routes'][0]['sections'][0]['departure']['time']))

'0:34:00'

Travel time

## Now access information for a sample of selected origin-destination points

In [15]:
# sample points lat-longs

df = pd.read_csv('https://raw.githubusercontent.com/CUSP2020PUI/Data/master/samplePoints.csv')
df.head()

,LocationID,lon_pickup,lat_pickup,LocationID2,lon_dropoff,lat_dropoff
0,1,-74.174000,40.691831,145,-73.948891,40.745379
1,2,-73.831299,40.616745,203,-73.739473,40.657853
2,3,-73.847422,40.864474,77,-73.895364,40.666559
3,4,-73.976968,40.723752,158,-74.008984,40.735035
4,5,-74.188484,40.552659,123,-73.964334,40.599954


In [16]:
len(df) ##one random point per each zip code

263

Function for getting driving time for a dataframe of above type

In [17]:
def driving(file, apiKey): 
    '''
    Input: the file that contain origin-destination lan/long information; keys from Map
    Output: list of driving times corresponding the input O-D pairs
    '''
    duration_list= []  # list for storing times
    for index, row in tqdm(file.iterrows()): # iterating through all rows of sample points
        lon_pickup = row['lon_pickup']  
        lat_pickup = row['lat_pickup']
        lon_dropoff = row['lon_dropoff']
        lat_dropoff = row['lat_dropoff']
        
        # specifying origin-desination lat-longs in the endpoint and making request
        url = str('https://router.hereapi.com/v8/routes?transportMode=car&origin={},{}&destination={},{}&return=summary&apikey={}'.format(lat_pickup, lon_pickup, lat_dropoff, lon_dropoff, apiKey))
        data = urllib.urlopen(url).read().decode('utf-8')
        data = json.loads(data)
        
        # append travel time to 'duration' list, nan if no travel time retrieved
        try:    
            duration = data['routes'][0]['sections'][0]['summary']['duration']
        except:
            duration = np.nan
            
        duration_list.append(duration)

    return duration_list  

Passing 10 random O-D pairs

In [18]:
sample = df.sample(10, random_state=2020)
sample.reset_index(drop=True, inplace=True)

sample

,LocationID,lon_pickup,lat_pickup,LocationID2,lon_dropoff,lat_dropoff
0,110,-74.128342,40.545780,254,-73.858948,40.882157
1,51,-73.828264,40.873973,92,-73.828859,40.761102
2,166,-73.961764,40.809457,234,-73.990458,40.740337
3,207,-73.899353,40.763986,248,-73.872289,40.834165
4,195,-74.009178,40.675549,68,-73.999917,40.748428
5,251,-74.125348,40.616880,114,-73.997380,40.728340
6,71,-73.937966,40.644288,118,-74.132979,40.586555
7,139,-73.744234,40.677098,202,-73.949952,40.761900
8,76,-73.876821,40.660935,240,-73.881978,40.894599
9,165,-73.956825,40.620924,258,-73.855767,40.688721


Calling the function to get responses from API

In [19]:
sample['driving_time'] = driving(sample, apiKey)
sample

10it [00:01,  8.88it/s]


,LocationID,lon_pickup,lat_pickup,LocationID2,lon_dropoff,lat_dropoff,driving_time
0,110,-74.128342,40.545780,254,-73.858948,40.882157,3877
1,51,-73.828264,40.873973,92,-73.828859,40.761102,1252
2,166,-73.961764,40.809457,234,-73.990458,40.740337,1102
3,207,-73.899353,40.763986,248,-73.872289,40.834165,1139
4,195,-74.009178,40.675549,68,-73.999917,40.748428,1040
5,251,-74.125348,40.616880,114,-73.997380,40.728340,1824
6,71,-73.937966,40.644288,118,-74.132979,40.586555,1940
7,139,-73.744234,40.677098,202,-73.949952,40.761900,2066
8,76,-73.876821,40.660935,240,-73.881978,40.894599,2350
9,165,-73.956825,40.620924,258,-73.855767,40.688721,1724


### NYC Open data API: 
NYC open data offers API for almost every available data from crime, housing, 311, taxi etc.
Also its free for the most part and no account needed

https://opendata.cityofnewyork.us/

#### Example: Endpoint for 2018 taxi data
https://data.cityofnewyork.us/resource/t29m-gskq.json

In [20]:
# some parameters to filter the data. We can specify as many parameters as columns in the table.
# reference: https://dev.socrata.com/foundry/data.cityofnewyork.us/t29m-gskq

# extracting info for a specific O-D pair corresponding to near E 35th st as pick up and Newark airport as drop off
# this is around the same pick-up and drop-off location we used to get driving time from HereMap API
parameter = {'pulocationid':162, 'dolocationid':1}



making API request

'requests' is another package of making web requests. Highly uselful in the case of REST APIs and we can provide parameters together in one line of code. We'll use the 'get' method to retrieve data

In [21]:
url =  "https://data.cityofnewyork.us/resource/t29m-gskq.json"
r = requests.get(url = url, params=parameter)
data = r.json()

fisrt three returned observations

In [22]:
data[:3]

[{'vendorid': '1',
  'tpep_pickup_datetime': '2018-02-14T13:10:19.000',
  'tpep_dropoff_datetime': '2018-02-14T13:53:15.000',
  'passenger_count': '1',
  'trip_distance': '17.40',
  'ratecodeid': '3',
  'store_and_fwd_flag': 'N',
  'pulocationid': '162',
  'dolocationid': '1',
  'payment_type': '1',
  'fare_amount': '68.5',
  'extra': '0',
  'mta_tax': '0',
  'tip_amount': '12',
  'tolls_amount': '10.5',
  'improvement_surcharge': '0.3',
  'total_amount': '91.3'},
 {'vendorid': '2',
  'tpep_pickup_datetime': '2018-02-14T14:25:43.000',
  'tpep_dropoff_datetime': '2018-02-14T15:05:18.000',
  'passenger_count': '2',
  'trip_distance': '17.40',
  'ratecodeid': '3',
  'store_and_fwd_flag': 'N',
  'pulocationid': '162',
  'dolocationid': '1',
  'payment_type': '1',
  'fare_amount': '68',
  'extra': '0',
  'mta_tax': '0',
  'tip_amount': '0',
  'tolls_amount': '16.1',
  'improvement_surcharge': '0.3',
  'total_amount': '84.4'},
 {'vendorid': '1',
  'tpep_pickup_datetime': '2018-02-14T14:38:48

Note that the returned request contains information reagrding pick up and drop off times, distance covered, fare amount, number of passengers etc. Essentially, it returns all info provided by TLC in any csv file.

Let's get travel time from first observation

In [23]:
# travel time in seconds
parser.parse(data[0]['tpep_dropoff_datetime']) - parser.parse(data[0]['tpep_pickup_datetime'])

datetime.timedelta(seconds=2576)

Note the travel time returned is around 40 minutes compared to around 28 minutes from driving time retrieved from HereMap API before. Also note time can vary based on the pick up times, the above time of 40 min correspond to pick up around 7pm.



#### get yellow taxi times for O-D pairs as before

#### Note: taxi data can be full of absurd values in terms of travel times/fares/distances. So we need to filter out all noise while we extract. 
#### We'll make the same checks for filtering as we did in the taxi data lab

1. Get rid of observations with travel time > 1 min and < 100 min
2. Speed (distance/time) should be > 2mph and <80mph
3. Fare should be < 300 USD and > 2.5 USD
4. distance traveled < 100 and > 0.3

In [24]:
def taxiTime(df):
    
    times = [] # list for storing travel times
    
    for index, row in df.iterrows(): # iterating through all rows of sample points
        
        # specify parameters for making request
        parameters = {'pulocationid':int(row['LocationID']), 'dolocationid':int(row['LocationID2'])}
        
        url =  "https://data.cityofnewyork.us/resource/t29m-gskq.json"
        r = requests.get(url = url, params=parameters)
        data = r.json()
        
        travelTime = []
        
        for obs in data: # iterating through each returned observation for the returned data 
            
            # making sanity checks and appending times to 'traveTime' list
            try:
                time = parser.parse(obs['tpep_dropoff_datetime']) - parser.parse(obs['tpep_pickup_datetime'])
                time = time.total_seconds()
                fare = float(obs['fare_amount'])
                distance = float(obs['trip_distance'])
                speed = distance/(time/3600)

                if (time < 6000 and time > 60 and fare < 200  
                    and fare > 2.5 and fare < 300 and speed < 80  and speed > 2 and distance > 0.3 and 
                    distance < 100):
                    travelTime.append(time)
                    
            except: 
                pass
            
        # now appending the mean of travel times retrieved above to the 'times' list
    
        times.append(np.mean(travelTime))
        
        
    return times

In [25]:
sample['TLC_avgtime'] = taxiTime(sample)
sample

C:\Applications\Anaconda\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Applications\Anaconda\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


,LocationID,lon_pickup,lat_pickup,LocationID2,lon_dropoff,lat_dropoff,driving_time,TLC_avgtime
0,110,-74.128342,40.545780,254,-73.858948,40.882157,3877,NaN
1,51,-73.828264,40.873973,92,-73.828859,40.761102,1252,1398.000000
2,166,-73.961764,40.809457,234,-73.990458,40.740337,1102,1834.531062
3,207,-73.899353,40.763986,248,-73.872289,40.834165,1139,NaN
4,195,-74.009178,40.675549,68,-73.999917,40.748428,1040,2149.000000
5,251,-74.125348,40.616880,114,-73.997380,40.728340,1824,NaN
6,71,-73.937966,40.644288,118,-74.132979,40.586555,1940,3126.000000
7,139,-73.744234,40.677098,202,-73.949952,40.761900,2066,NaN
8,76,-73.876821,40.660935,240,-73.881978,40.894599,2350,3203.666667
9,165,-73.956825,40.620924,258,-73.855767,40.688721,1724,2622.000000


In [38]:
## get latitude and longitude from returned json request

